<a href="https://colab.research.google.com/drive/1Ib3QcE9TgUZHuYXgepwk9FS-eSbDniQ8?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Automatic Prompt Engineer (APE)

APE is a method for automatically generating and optimizing prompts for language models.

### Key points:

* **Core concept:** Use AI to create and refine prompts, reducing manual engineering effort.

* **Process:**
  * Generate candidate prompts
  * Evaluate prompt performance
  * Iteratively optimize prompts

* **Components:**
  * Prompt generator
  * Performance evaluator
  * Optimization algorithm

* **Advantages:**
  * Discovers effective prompts automatically
  * Adapts to different tasks and model architectures

* **Applications:**
  * Task-specific prompt optimization
  * Improving model performance across various domains

* **Implementation:**
  * Define task and evaluation metrics
  * Use large language models to generate initial prompts
  * Apply optimization techniques (e.g., genetic algorithms, gradient-based methods)

* **Challenges:**
  * Balancing exploration and exploitation in prompt space
  * Ensuring generated prompts are interpretable and safe
  
* **Variations:**
  * Multi-task APE: Optimize prompts for multiple related tasks
  * Constrained APE: Generate prompts within specific guidelines or structures

In [1]:
!pip install -q -U langchain-groq==0.2.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.6 MB/s eta 0:00:00


In [6]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

In [3]:
import os
import getpass

### 🔑 Provide Groq API Key

- [Groq API Key](https://console.groq.com/keys)


In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [5]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5
)

In [7]:
import re

# APE Prompt Generation
ape_generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI specialized in creating effective prompts for language models."),
        ("human", """Task: Create a prompt that will help a language model perform the following task effectively:

          {task}

          Generate 3 different prompts for this task. Each prompt should be designed to elicit a high-quality

          response from a language model. Consider different angles, formats, and instructions that might lead to better results.

          Your response should be in the following format:

          Prompt 1:
          [Your first prompt here]

          Prompt 2:
          [Your second prompt here]

          Prompt 3:
          [Your third prompt here]

          Generated Prompts:""")
    ]
)

# APE Evaluation
ape_evaluation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI specialized in evaluating the effectiveness of prompts for language models."),
        ("human", """Evaluate the following prompts for their effectiveness in accomplishing this task:

          Task: {task}

          {prompts}

          For each prompt, provide a score from 1-10 and a brief explanation of its strengths and weaknesses.

          Consider factors such as clarity, specificity, and potential to elicit high-quality responses.

          Your evaluation should be in the following format:

          Prompt 1:
          Score: [score]/10
          Evaluation: [Your evaluation here]

          Prompt 2:
          Score: [score]/10
          Evaluation: [Your evaluation here]

          Prompt 3:
          Score: [score]/10
          Evaluation: [Your evaluation here]

          Your evaluation:""")
    ]
)

def generate_prompts(task):
    chain = ape_generation_prompt | llm
    return chain.invoke({"task": task}).content

def evaluate_prompts(task, prompts):
    chain = ape_evaluation_prompt | llm
    return chain.invoke({"task": task, "prompts": prompts }).content

def test_prompt(prompt, task):
    test_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", "You are an AI assistant completing a given task."),
          ("human", "{prompt}\n\nTask: {task}")
      ]
    )
    chain = test_prompt | llm
    return chain.invoke({"task": task, "prompt": prompt }).content

def parse_prompts(generated_prompts):
    prompts = re.findall(
        r"Prompt \d+:\n(.*?)(?=\n\nPrompt \d+:|$)", generated_prompts, re.DOTALL
    )
    return [prompt.strip() for prompt in prompts if prompt.strip()]


def parse_scores(evaluation):
    scores = re.findall(r"Score: (\d+)/10", evaluation)
    return [int(score) for score in scores if score.isdigit()]


def ape_process(task, iterations=2):
    best_prompt = ""
    best_score = 0

    for i in range(iterations):
        print(f"Iteration {i+1}")

        # Generate prompts
        generated_prompts = generate_prompts(task)
        display(Markdown(f"**Generated Prompts:**\n{generated_prompts}"))

        # Evaluate prompts
        evaluation = evaluate_prompts(task, generated_prompts)
        display(Markdown(f"**Prompt Evaluation:**\n{evaluation}"))

        # Parse prompts and scores
        prompts = parse_prompts(generated_prompts)
        scores = parse_scores(evaluation)

        # Ensure we have valid prompts and scores
        if prompts and scores:
            # Make sure we have the same number of prompts and scores
            min_length = min(len(prompts), len(scores))
            prompts = prompts[:min_length]
            scores = scores[:min_length]

            if max(scores) > best_score:
                best_score = max(scores)
                best_prompt = prompts[scores.index(max(scores))]

            print(f"Best prompt so far (score {best_score}/10):")
            print(best_prompt)
        else:
            print("Failed to generate valid prompts or scores in this iteration.")

        print()

    # If we didn't find a good prompt, use a default one
    if not best_prompt:
        best_prompt = f"Please {task}"
        print("Using default prompt due to generation issues.")

    # Test the best prompt
    final_result = test_prompt(best_prompt, task)
    return best_prompt, final_result

In [8]:
# Example usage
task = "Explain the concept of quantum entanglement to a 10-year-old."

best_prompt, final_result = ape_process(task)

display(
    Markdown(
        f"""**Final Best Prompt:**
{best_prompt}

**Final Result:**
{final_result}"""
    )
)


Iteration 1


**Generated Prompts:**
Here are three different prompts to help a language model explain quantum entanglement to a 10-year-old:

Prompt 1:
Explain quantum entanglement in a way that a 10-year-old could understand. Use simple language and avoid technical jargon. Imagine you're telling a friend about a cool science concept, and make it sound exciting and fun!

Prompt 2:
Write a story about two toy cars that are "entangled" in a way that's similar to quantum entanglement. How do they behave when they're connected, and what happens when they're separated? Use this analogy to help explain the concept of entanglement in a way that's relatable and easy to grasp.

Prompt 3:
Imagine you have a special kind of walkie-talkie that lets you talk to your friend who's on the other side of the world, but it works in a weird way. When you say something into the walkie-talkie, your friend hears it instantly, even if you're thousands of miles apart. How does it work? Use this thought experiment to help explain the concept of quantum entanglement, and make sure to highlight the strange and amazing implications of this phenomenon.

**Prompt Evaluation:**
Here are my evaluations for each prompt:

Prompt 1:
Score: 7/10
Evaluation: This prompt is clear and concise, and it provides a good starting point for the language model to explain quantum entanglement in simple language. The instruction to use "simple language and avoid technical jargon" is helpful, and the phrase "make it sound exciting and fun" encourages the model to be engaging and enthusiastic. However, the prompt is quite general and doesn't provide a specific framework or analogy for the model to work with, which might lead to a response that is more of a general explanation rather than a concrete example.

Prompt 2:
Score: 9/10
Evaluation: This prompt is excellent! It provides a specific and relatable analogy that can help explain the concept of entanglement in a way that's easy to grasp. The idea of two toy cars being "entangled" is a great way to illustrate the phenomenon, and the instruction to use this analogy to explain entanglement provides a clear framework for the model to work with. The prompt also encourages the model to think creatively and come up with a story that's engaging and fun. The only potential weakness is that the analogy might not fully capture the complexities of quantum entanglement, but overall, this prompt is well-designed.

Prompt 3:
Score: 8.5/10
Evaluation: This prompt is very effective! It provides a thought-provoking analogy that can help explain the concept of entanglement in a way that's easy to understand. The idea of a special walkie-talkie that lets you talk to your friend instantly, no matter the distance, is a great way to illustrate the phenomenon of entanglement. The prompt also encourages the model to think creatively and come up with a explanation that's engaging and fun. The only potential weakness is that the analogy might be a bit too complex for a 10-year-old, and the model might need to simplify the explanation further. However, overall, this prompt is well-designed and has the potential to elicit a high-quality response.

Best prompt so far (score 9/10):
Write a story about two toy cars that are "entangled" in a way that's similar to quantum entanglement. How do they behave when they're connected, and what happens when they're separated? Use this analogy to help explain the concept of entanglement in a way that's relatable and easy to grasp.

Iteration 2


**Generated Prompts:**
Here are three prompts designed to help a language model explain quantum entanglement to a 10-year-old:

Prompt 1:
Explain quantum entanglement in a way that a 10-year-old can understand, using simple language and relatable examples. Imagine you're explaining it to a friend who has never heard of it before. Make sure to include a fun analogy or metaphor to help them grasp the concept.

Prompt 2:
Write a story about two toy cars, "Racing Ruby" and "Speedy Sam," that are connected by a special kind of invisible string. When something happens to one car, it instantly affects the other car, even if they're on opposite sides of the room. Use this story to explain the concept of quantum entanglement, focusing on the idea that particles can be connected in a way that lets them affect each other instantly, no matter how far apart they are.

Prompt 3:
Imagine you're a scientist who has just discovered a magical box that can connect two objects in a special way. When you put a toy in the box with a partner toy, they become "entangled" and can affect each other in strange and amazing ways. Write a report to a 10-year-old describing the magical box and its powers, using simple language and examples to explain how it works. Be sure to include some cool and surprising facts about the box and its effects on the entangled objects.

**Prompt Evaluation:**
Here are my evaluations for each prompt:

Prompt 1:
Score: 8/10
Evaluation: This prompt is clear and concise, providing a good foundation for the language model to explain quantum entanglement in a simple and relatable way. The instruction to use a fun analogy or metaphor is particularly helpful, as it encourages the model to be creative and engaging. However, the prompt could be improved by providing more specific guidance on what kind of examples or analogies would be most effective for a 10-year-old.

Prompt 2:
Score: 9/10
Evaluation: This prompt is highly effective, as it uses a storytelling approach to explain a complex concept in a way that is both engaging and easy to understand. The use of relatable characters and a concrete scenario helps to make the concept of entanglement more accessible to a 10-year-old. The prompt also provides a clear structure for the language model to follow, making it more likely to produce a high-quality response. The only potential weakness is that the prompt may be a bit too focused on the toy cars, which could limit the model's ability to provide additional context or explanations.

Prompt 3:
Score: 7/10
Evaluation: This prompt is a good attempt to explain quantum entanglement using a fictional scenario, but it could be improved in several ways. The instruction to write a report to a 10-year-old is a bit vague, and the prompt could benefit from more specific guidance on what kind of information to include and how to present it. Additionally, the magical box analogy, while creative, may not be the most effective way to explain entanglement, as it may be perceived as too fantastical or misleading. However, the prompt does provide a clear structure and encourages the model to use simple language and examples, which are important for explaining complex concepts to a 10-year-old.

Best prompt so far (score 9/10):
Write a story about two toy cars that are "entangled" in a way that's similar to quantum entanglement. How do they behave when they're connected, and what happens when they're separated? Use this analogy to help explain the concept of entanglement in a way that's relatable and easy to grasp.



**Final Best Prompt:**
Write a story about two toy cars that are "entangled" in a way that's similar to quantum entanglement. How do they behave when they're connected, and what happens when they're separated? Use this analogy to help explain the concept of entanglement in a way that's relatable and easy to grasp.

**Final Result:**
Once upon a time, in a toy box filled with all sorts of fun and colorful toys, there lived two best friend toy cars, Zoom and Zoomster. They were identical, shiny, and loved to speed around the room, racing each other and having the best of times.

One day, a curious scientist, Dr. Toybox, discovered that Zoom and Zoomster had a special connection. Whenever they were placed together, they would start to behave in a very strange and amazing way.

When Zoom moved to the left, Zoomster would instantly move to the right, even if they were on opposite sides of the room! It was as if they were connected by an invisible string. But here's the really cool part: when Zoomster moved to the right, Zoom would instantly move to the left, even if they were still on opposite sides of the room!

Dr. Toybox was fascinated by this phenomenon and decided to test it further. She placed Zoom and Zoomster in separate boxes, so they couldn't see or touch each other. Then, she had someone move Zoom to the left while someone else moved Zoomster to the right.

Guess what? Even though they were separated by miles, Zoom and Zoomster still reacted to each other's movements! When Zoom moved left, Zoomster moved right, and vice versa. It was as if they were still connected by that invisible string.

This is similar to what happens with quantum entanglement. When two tiny particles, like atoms or electrons, are connected in a special way, they can affect each other even if they're really far apart. It's like they're "talking" to each other in a secret language that's faster than the speed of light.

Imagine you have two toy cars, just like Zoom and Zoomster. If they're entangled, whenever one car moves, the other car will move instantly, no matter where it is in the world. It's like they're connected by a magic string that lets them communicate with each other.

But here's the really cool part: even if you separate the cars and take them to opposite sides of the universe, they'll still be connected. If you make one car spin around, the other car will spin around too, even if it's on the other side of the universe!

So, to sum it up, quantum entanglement is like having two toy cars that are connected in a special way. When they're together, they can affect each other instantly, and even when they're really far apart, they can still "talk" to each other in a secret language. It's a really cool and weird phenomenon that scientists are still trying to understand better!

I hope that helps you understand quantum entanglement, kiddo!